# Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [3]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [2]:
df = df.drop(columns=['salary', 'salary_currency'])

In [ ]:
from sklearn.model_selection import train_test_split

y = df['salary_in_usd']
X = df.drop('salary_in_usd', axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
)

X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test,
    test_size=0.5,
    random_state=42
)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_features = ['experience_level', 'employment_type', 'job_title',
                'employee_residence', 'company_location', 'company_size']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ]
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

reg_mape = mean_absolute_percentage_error(y_test, y_test_pred)
reg_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print('MAPE: ', reg_mape)
print('RMSE: ', reg_rmse)

MAPE:  0.3694070606254982
RMSE:  51730.9424060791


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
import time
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300, 500],
    'gamma': [0, 0.1, 0.3, 0.5],
    'reg_alpha': [0, 0.1, 0.3, 0.5],
    'reg_lambda': [1, 1.5, 2, 2.5]
}

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomizedSearchCV(XGBRegressor(),
                                     param_distributions=params, scoring='neg_mean_squared_error',
                                     n_iter=50, cv=3, n_jobs=-1, random_state=42))
])
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
fit_time = end_time - start_time

best_XGB_params = model.named_steps['regressor'].best_params_
print('Best XGB params:', best_XGB_params)

start_time = time.time()
y_test_pred = model.predict(X_test)
end_time = time.time()
predict_time = end_time - start_time

print('fit_time: ', fit_time)
print('predict_time: ', predict_time)

Best XGB params: {'reg_lambda': 2.5, 'reg_alpha': 0, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1}
fit_time:  43.36429262161255
predict_time:  0.008548974990844727


In [ ]:
xgb_mape = mean_absolute_percentage_error(y_test, y_test_pred)
xgb_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print('MAPE: ', xgb_mape)
print('RMSE: ', xgb_rmse)

MAPE:  0.3490324043384218
RMSE:  50587.515503679715


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.3 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

params = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [100, 200, 500, 1000],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128, 256]
}

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomizedSearchCV(CatBoostRegressor(verbose=0),
                                     param_distributions=params, scoring='neg_mean_squared_error',
                                     n_iter=50, cv=3, n_jobs=-1, random_state=42))
])

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
fit_time = end_time - start_time

best_catboost_params = model.named_steps['regressor'].best_params_
print('Best catboost params:', best_catboost_params)

start_time = time.time()
y_test_pred = model.predict(X_test)
end_time = time.time()
predict_time = end_time - start_time

print('fit_time: ', fit_time)
print('predict_time: ', predict_time)

Best catboost params: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 8, 'border_count': 64}
fit_time:  227.4683334827423
predict_time:  0.009087562561035156


In [ ]:
catboost_mape = mean_absolute_percentage_error(y_test, y_test_pred)
catboost_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print('MAPE: ', catboost_mape)
print('RMSE: ', catboost_rmse)

MAPE:  0.3416595174155115
RMSE:  50123.91862246364


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [ ]:
from catboost import Pool


cat_features = ['experience_level', 'employment_type', 'job_title',
                'employee_residence', 'company_location', 'company_size']

params = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [100, 200, 500, 1000],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128, 256]
}

model = RandomizedSearchCV(
    estimator=CatBoostRegressor(verbose=0, cat_features=cat_features),
    param_distributions=params,
    scoring='neg_mean_squared_error',
    n_iter=50,
    cv=3,
    n_jobs=-1,
    random_state=42
)

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
fit_time = end_time - start_time

best_params = model.best_params_
print('Best params:', best_params)

final_model = CatBoostRegressor(**best_params, verbose=0)
final_model.fit(train_pool, eval_set=val_pool, use_best_model=True)

start_time = time.time()
y_test_pred = final_model.predict(X_test)
end_time = time.time()
predict_time = end_time - start_time

print('fit_time: ', fit_time)
print('predict_time: ', predict_time)

catboost_mape = mean_absolute_percentage_error(y_test, y_test_pred)
catboost_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print('MAPE: ', catboost_mape)
print('RMSE: ', catboost_rmse)

Best params: {'learning_rate': 0.01, 'l2_leaf_reg': 7, 'iterations': 1000, 'depth': 8, 'border_count': 64}
fit_time:  699.0497632026672
predict_time:  0.0032341480255126953
MAPE:  0.370827286517799
RMSE:  50192.33335423006


**Ответ:** 3мин 42сек против 11мин 47сек. А качество почти не изменилось, даже чуть-чуть ухудшилось

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [ ]:
from lightgbm import LGBMRegressor


params = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500, 1000],
    'min_child_samples': [20, 30, 40],
    'num_leaves': [31, 50, 70],
}

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomizedSearchCV(LGBMRegressor(random_state=42, force_row_wise=True, verbose=-1),
                                     param_distributions=params, scoring='neg_mean_squared_error',
                                     n_iter=50, cv=3, n_jobs=-1, random_state=42))
])

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
fit_time = end_time - start_time

best_LGBM_params = model.named_steps['regressor'].best_params_
print('Best LGBM params:', best_LGBM_params)

start_time = time.time()
y_test_pred = model.predict(X_test)
end_time = time.time()
predict_time = end_time - start_time

print('fit_time: ', fit_time)
print('predict_time: ', predict_time)

Best LGBM params: {'num_leaves': 31, 'n_estimators': 1000, 'min_child_samples': 20, 'max_depth': 3, 'learning_rate': 0.05}
fit_time:  31.354214668273926
predict_time:  0.018389463424682617


In [ ]:
lgbm_mape = mean_absolute_percentage_error(y_test, y_test_pred)
lgbm_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

print('MAPE: ', lgbm_mape)
print('RMSE: ', lgbm_rmse)

MAPE:  0.3425965612350088
RMSE:  50608.75562090897


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [ ]:
print('Best XGB params:', best_XGB_params)
print('MAPE: ', xgb_mape)
print('RMSE: ', xgb_rmse)
print('time: 44сек')
print()

print('Best catboost params:', best_catboost_params)
print('MAPE: ', catboost_mape)
print('RMSE: ', catboost_rmse)
print('time: 3мин 42сек')
print()

print('Best LGBM params:', best_LGBM_params)
print('MAPE: ', lgbm_mape)
print('RMSE: ', lgbm_rmse)
print('time: 32сек')

Best XGB params: {'reg_lambda': 2.5, 'reg_alpha': 0, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1}
MAPE:  0.3490324043384218
RMSE:  50587.515503679715
time: 44сек

Best catboost params: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 8, 'border_count': 64}
MAPE:  0.3416595174155115
RMSE:  50123.91862246364
time: 3мин 42сек

Best LGBM params: {'num_leaves': 31, 'n_estimators': 1000, 'min_child_samples': 20, 'max_depth': 3, 'learning_rate': 0.05}
MAPE:  0.3425965612350088
RMSE:  50608.75562090897
time: 32сек


**Ответ:**
XGBoost: Использует небольшую глубину деревьев - 3, небольшое количество деревьев - 200, среднюю скорость обучения - 0.1

CatBoost: Использует высокую глубину деревьев - 8, умеренное количество итераций - 500, небольшую скорость обучения - 0.05

LightGBM: Использует небольшую глубину деревьев - 3, большое количество итераций - 1000, небольшую скорость обучения - 0.05, большое количество листьев - 31, минимальное количество образцов для разделения - 20
1. Лучшее качество: CatBoost
2. Худшее качество: LGBM
3. Самое быстрое обучение: LGBM
4. Самое медленное обучение: CatBoost

Если у нас есть много лишнего времени, то лучше использовать CatBoost, если его мало, то LGBM, а XGB хороший баланс качества и временем обучения